## import 

In [844]:
import time
import hashlib
import requests
import xmltodict, json
import uuid 
import re
import datetime
import hashlib
import requests
import json
from pprint import pprint
import pymongo
from pymongo import MongoClient
import logging
import threading
import time
import redis
import progressbar
client=MongoClient('mongodb://localhost:27017')
db=client.test
hotel=db.jiudian_hotels
room=db.jiudian_room

AID = 328727
SID =  810129
UUID=uuid.uuid1()
ICODE='e6fb53910a134cdd96fc4ba30f864c7d'
API_URL = 'openapi.ctrip.com'
KEY =  'E0F33944-A048-45F7-AA91-A55196051B91'
TOKEN_KEY =  'f017b34784be441780a8cc79ccd4a9ed'
type(UUID)

uuid.UUID

In [802]:
class XiechengTask():
    abstract = True
    _mdb = None
    _rdb = None

    @property
    def mdb(self):
        if self._mdb is None:
            _mc = MongoClient('mongodb://localhost:27017')
            self._mdb = _mc["test"]
        return self._mdb

    @property
    def rdb(self):
        if self._rdb is None:
            self._rdb = redis.StrictRedis(host='127.0.0.1',
                                          port='6379',
                                          db=0)
        return self._rdb
m=XiechengTask()

### 获取、更新携程access token

In [913]:
def token_fetch_and_update():
    """获取、更新携程access token,存储于radis中,失效后可再次获取
    """
    try:
        token = 'https://openserviceauth.ctrip.com/openserviceauth/authorize.ashx?AID={0}&SID={1}&KEY={2}'.format(
            AID, SID, TOKEN_KEY)
        token_result = requests.post(token)
        Refresh_Token = token_result.json()['Refresh_Token']

        Atoken = 'https://openserviceauth.ctrip.com/openserviceauth/refresh.ashx?AID={0}&SID={1}&refresh_token={2}'.format(
            AID, SID, Refresh_Token)
        Atoken_result = requests.post(Atoken)
        Access_Token = Atoken_result.json()['Access_Token']
    except KeyError:
        return ['token_fetch_and_update retry']

    m.rdb.hset('xiecheng_token', 'access_token', Access_Token)
    m.rdb.hset('xiecheng_token', 'expire_time',
                  datetime.datetime.now() + datetime.timedelta(seconds=600))
    return ['xiecheng_access_token update is OK!']
# token_fetch_and_update()

## 查询酒店全量ID列表

In [915]:
from __future__ import absolute_import, unicode_literals
def city_get_full_hotel_ids(city, page_index=1):
    """根据城市ID，查询酒店全量ID列表，并调用静态信息接口
    """
    Access_Token = m.rdb.hget('xiecheng_token', 'access_token').decode('UTF-8')
#     print(Access_Token)
    url_html = 'http://openservice.ctrip.com/OpenService/ServiceProxy.ashx?AID={0}&SID={1}&ICODE={2}&UUID={3}&Token={4}&mode=1&format=json'.format(
        AID, SID, ICODE, UUID, Access_Token)
    data = {
        "City": city,
        "PageIndex": str(page_index),
        "PageSize": "5000"
    }

    try:
        requests_result = requests.post(url_html, data=data)
        resp_data = requests_result.json()
        total = resp_data['Total']
        if int(total) == 0:
            return ['city', city, 'total 0']
        hotel_ids = resp_data['HotelIDs'].split(',')
    except KeyError as e:
        if str(datetime.datetime.now()) > (self.rdb.hget(
                'xiecheng_token', 'expire_time')).decode('UTF-8'):
            token_fetch_and_update()
        city_get_full_hotel_ids(city, page_index)
        return['city', city, 'EOFError']

    # if page_index * 5000 < total:
    #     city_get_full_hotel_ids.apply_async(args=[city, page_index + 1])

    # for hotel_ids_index in range(len(hotel_ids) // 10 + 1):
    #     hotel_ids_message_fetch.delay(
    # hotel_ids[hotel_ids_index * 10:(hotel_ids_index + 1) * 10])  # async
#     return [city, page_index, len(hotel_ids), 'is over']
city_get_full_hotel_ids(22019)

['city', 22019, 'total 0']

### 获取所有城市 ID

In [819]:
import time
import hashlib
import requests
import xmltodict, json
import uuid 
import re
import datetime
import hashlib
import requests
import json
from pprint import pprint
import pymongo
from pymongo import MongoClient
import logging
import threading
import time
import redis
import progressbar
client=MongoClient('mongodb://localhost:27017')
db=client.test
hotel=db.jiudian_hotels
room=db.jiudian_room

AID = 328727
SID =  810129
UUID=uuid.uuid1()
ICODE='e6fb53910a134cdd96fc4ba30f864c7d'
API_URL = 'openapi.ctrip.com'
KEY =  'E0F33944-A048-45F7-AA91-A55196051B91'
TOKEN_KEY =  'f017b34784be441780a8cc79ccd4a9ed'


def city_get_full_hotel_ids(city, page_index=1):
    """根据城市ID，查询酒店全量ID列表，并调用静态信息接口
    """
    token = 'https://openserviceauth.ctrip.com/openserviceauth/authorize.ashx?AID={0}&SID={1}&KEY={2}'.format(
        AID, SID, TOKEN_KEY)
    token_result = requests.post(token)
    Refresh_Token = token_result.json()['Refresh_Token']

    Atoken = 'https://openserviceauth.ctrip.com/openserviceauth/refresh.ashx?AID={0}&SID={1}&refresh_token={2}'.format(
        AID, SID, Refresh_Token)
    Atoken_result = requests.post(Atoken)
    Access_Token = Atoken_result.json()['Access_Token']    
    url_html = 'http://openservice.ctrip.com/OpenService/ServiceProxy.ashx?AID={0}&SID={1}&ICODE={2}&UUID={3}&Token={4}&mode=1&format=json'.format(
        AID, SID, ICODE, UUID, Access_Token)
    data = {
        "City": city,
        "PageIndex": str(page_index),
        "PageSize": "5000"
    }
    try:
        requests_result = requests.post(url_html, data=data)
        resp_data = requests_result.json()
        total = resp_data['Total']
        return int(total)
    except KeyError :
        return city_get_full_hotel_ids(city,page_index)


def full_citys_fetch():
    with open(r"/home/fanjindong/notebook/celery/celery_demo/celery_app/携程国内城市.xml") as f:
        data = f.read()
    data_json = xmltodict.parse(data)
    items = data_json['CityDetails']['CityDetail']
    citys = {}
    for item in items:
        city = int(item['City'])
        values=city_get_full_hotel_ids(city)  # async
        citys[city]=values
        print(city,values)
        time.sleep(1)
    return citys

if __name__ == "__main__":
    citys=full_citys_fetch()

1 14665
2 10406
3 4780
4 12688
5 5497
6 3604
7 6494
10 11857
11 656
12 4365
13 1602
14 3392
15 966
16 416
17 5387
18 0
19 2182
20 499
21 0
22 784
23 1236
24 786
25 4802
26 648
27 1727
28 8431
30 8469
31 1843
32 11090
33 1598
34 None
35 0
36 3882
37 4615
38 2650
39 1662
40 107
41 1319
42 1555
43 3193
44 211
45 201
46 49
47 44
48 125
49 136
50 73
51 0
52 273
53 35
54 63
55 185
56 67
57 298
58 1122
59 91
82 721
83 1080
84 144
85 264
86 None
87 215
90 1136
91 671
92 125
93 0
94 1052
95 678
96 280
97 3
98 3
99 1467
100 2207
103 1514
104 1055
105 2655
106 0
108 200
109 171
110 449
111 718
112 574
117 36
118 155
119 253
123 66
124 2237
125 23
126 11
127 39
128 None
129 290
130 0
131 143
132 149
133 0
134 105
135 113
136 573
137 597
139 594
140 430
141 958
142 0
143 393
144 3504
145 0
147 3701
149 816
150 750
153 88
155 215
156 67
157 222
158 3209
159 1208
161 83
162 108
165 206
166 161
171 None
173 10
175 5
177 393
178 331
179 394
180 82
181 386
182 438
184 0
185 1064
186 429
187 0
188 0
189 

In [821]:
full_citys={k:v for k,v in citys.items() if v}

In [822]:
sum([v for _,v in full_citys.items()])

505373

## 酒店静态信息查询

In [372]:
def hotel_message_fetch(hotel_id, try_number=1):
    """酒店静态信息查询"""
    ping_url = 'http://{}/Hotel/OTA_HotelDescriptiveInfo.asmx'.format(API_URL)
    ts = int(time.time())
    secret = hashlib.md5(KEY.encode('utf8')).hexdigest().upper()
    sign = hashlib.md5('{0}{1}{2}{3}OTA_Ping'.format(
        ts, AID, secret, SID).encode('utf-8')).hexdigest().upper()

    request_xml = ('<?xml version="1.0" encoding="utf-8"?>'
                   '<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">'
                   '<soap:Body>'
                   '<Request xmlns="http://ctrip.com/">'
                   '<requestXML><![CDATA['
                   '<?xml version="1.0" encoding="utf-8"?>'
                   '<Request>'
                   '<Header AllianceID="{0}" SID="{1}" TimeStamp="{2}" RequestType="OTA_Ping" Signature="{3}" />'
                   '<HotelRequest>'
                   '<RequestBody xmlns:ns="http://www.opentravel.org/OTA/2003/05" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema">'
                   '<OTA_HotelDescriptiveInfoRQ Version="1.0" xsi:schemaLocation="http://www.opentravel.org/OTA/2003/05OTA_HotelDescriptiveInfoRQ.xsd" xmlns="http://www.opentravel.org/OTA/2003/05" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">'
                   '<HotelDescriptiveInfos>'
                   '<HotelDescriptiveInfo HotelCode="{4}" PositionTypeCode="502">'
#                    '<HotelInfo SendData="true"/>'
#                    '<FacilityInfo SendGuestRooms="true"/>'
#                    '<AreaInfo SendAttractions="true" SendRecreations="true"/>'
#                    '<ContactInfo SendData="true"/>'
#                    '<MultimediaObjects SendData="true"/>'
                   '</HotelDescriptiveInfo>'
                   
                   '<HotelDescriptiveInfo HotelCode="369782" PositionTypeCode="502">'
                   '<HotelInfo SendData="true"/>'
                   '<FacilityInfo SendGuestRooms="true"/>'
                   '<AreaInfo SendAttractions="true" SendRecreations="true"/>'
                   '<ContactInfo SendData="true"/>'
                   '<MultimediaObjects SendData="true"/>'
                   '</HotelDescriptiveInfo>'
                   
                   '</HotelDescriptiveInfos>'
                   '</OTA_HotelDescriptiveInfoRQ>'
                   '</RequestBody>'
                   '</HotelRequest>'
                   '</Request>]]>'
                   '</requestXML>'
                   '</Request>'
                   '</soap:Body>'
                   '</soap:Envelope>').format(AID, SID, ts, sign, hotel_id)

    encode_xml = request_xml.encode('utf-8')

    headers = {'Content-Type': 'text/xml; charset=utf-8',
               'Host': 'openapi.ctrip.com', 'Content-Length': len(encode_xml)}
    resp = requests.post(url=ping_url, data=encode_xml, headers=headers).text
    xmlt_data = xmltodict.parse(xmltodict.parse(resp)[
        'soap:Envelope']['soap:Body']['RequestResponse']['RequestResult'])
    # xml 转 dict
    xmlt_data = (json.loads(
        (json.dumps(xmlt_data, ensure_ascii=False, indent=2)).replace('@', '')))
#     print(json.dumps(xmlt_data, ensure_ascii=False, indent=2))
#     pprint(xmlt_data)
    return xmlt_data
    # dict 拆解 并去除’@‘
    dirty_datas = {}
    clean_datas = {}
    room_datas = {}

    def dict_open(self):
        """dict拆分"""
        if isinstance(self, dict):
            for key in self.keys():
                value = self[key]
                dirty_datas[key] = value
                dict_open(value)
        if isinstance(self, list):
            for item in self:
                dict_open(item)
    # 调用dict拆分function并赋值给dirty_datas
    dict_open(xmlt_data)

    def judgment(keys, data=dirty_datas, default=[]):
        try:
            if isinstance(keys, str):
                return data.get(keys, default)
            if isinstance(keys, list):
                if not len(keys) == 1:
                    if judgment(keys[0], data=data):
                        return judgment(keys[1:], data=judgment(keys[0], data=data))
                else:
                    return judgment(keys[-1], data=data)
        except KeyError as e:
            return []

    clean_datas['id'] = judgment('HotelCode')

    if (not clean_datas['id'])and try_number <= 3:
        # 酒店ID查询为空，重新查询（最大重试次数为3）
        hotel_message_fetch(
            (hotel_id, try_number + 1), countdown=30)
        return [hotel_id, 'message retries', try_number, '30s']
    elif (not clean_datas['id'])and try_number > 3:
        # 酒店ID查询为空，重试次数超过3次，忽略之.
        return [hotel_id, 'message not found']
    elif (not judgment('GuestRoom'))and clean_datas['id']:
        # 酒店ID非空，rooms字段为空，判断为下线酒店，执行数据库删除操作
#         self.mdb['jiudian_hotels'].delete_many({'id': clean_datas['id']})
        return [clean_datas['id'], 'delete basedata is over']

    clean_datas['name'] = judgment('HotelName')
    clean_datas['brandCode'] = judgment('BrandCode')
    clean_datas['cityCode'] = judgment('HotelCityCode')
    clean_datas['areaId'] = judgment('AreaID')

    items = judgment('Service')
    if not isinstance(items, list):
        items = [items]
    clean_datas['services'] = []
    for item in items:
        clean_datas['services'].append(judgment('Code', data=item))

    items = judgment('ThemeCategoryType')
    if not isinstance(items, list):
        items = [items]
    clean_datas['themes'] = []
    for item in items:
        if judgment('Code', data=item):
            clean_datas['themes'].append(judgment('Code', data=item))

    clean_datas['images'] = []
    items = judgment('ImageItem')
    if not isinstance(items, list):
        items = [items]
    for item in items:
        clean_datas['images'].append({
            'category': judgment('Category', data=item),
            'caption': judgment(["Description", 'Caption'], data=item),
            'url': judgment(["ImageFormat", 'URL'], data=item)
        })

    clean_datas['phones'] = []
    items = judgment('Phone')
    if not isinstance(items, list):
        items = [items]
    for item in items:
        if judgment('PhoneNumber', data=item):
            clean_datas['phones'].append(item['PhoneNumber'])

    clean_datas['descriptions'] = []
    items = judgment('TextItem')
    if not isinstance(items, list):
        items = [items]
    for item in items:
        if item.get('Category') and item.get('Description'):
            clean_datas['descriptions'].append(
                {'category': item['Category'], 'content': item['Description']})
    clean_datas['address'] = judgment('Address')
    clean_datas['lat'] = judgment('Latitude')
    clean_datas['lng'] = judgment('Longitude')
    clean_datas['lng'] = [clean_datas['lng'], clean_datas['lat']]
    clean_datas['categoryCodes'] = judgment(['SegmentCategory', 'Code'])
    clean_datas['isOnlineFranchise'] = judgment('IsOnlineFranchise')
    clean_datas['lastUpdated'] = judgment('LastUpdated')
    if judgment('WhenBuilt'):
        clean_datas['whenBuilt'] = judgment('WhenBuilt')
    dl = []
    items = judgment(['Policy', 'PolicyInfoCodes',
                      'PolicyInfoCode', 'Description'])
    if not isinstance(items, list):
        items = [items]
    for item in items:
        if isinstance(item, dict):
            if item.get('Text') and item.get('Name'):
                dl.append({'Text': judgment('Text', item),
                           'Name': judgment('Name', item)})
    clean_datas['policies'] = {
        "PolicyInfo": {
            "CheckOutTime": judgment(['Policy', 'PolicyInfo', 'CheckOutTime']),
            "CheckInTime": judgment(['Policy', 'PolicyInfo', 'CheckInTime'])
        },
        "PolicyInfoCodes": {
            "PolicyInfoCode": {
                "Description": dl,
                "Code": judgment(['Policy', 'PolicyInfoCodes', 'PolicyInfoCode', 'Code'])
            }
        }
    }
    clean_datas['refPoints'] = judgment('RefPoint')
    clean_datas['awards'] = {}
    items = judgment('Award')
    if not isinstance(items, list):
        items = [items]
    for item in items:
        if judgment('Provider', data=item):
            clean_datas['awards'][judgment('Provider', data=item)] = judgment(
                'Rating', data=item)
    clean_datas['tpa_Extensions'] = {
        "MasterSubHotelIDs": judgment('MasterSubHotelIDs'),
        "Roomquantity": judgment('Roomquantity'),
        "CityImportantMessage": judgment('CityImportantMessage'),
        "ThemeCategory": judgment('ThemeCategory')
    }

    # room
    rooms = judgment('GuestRoom')
    rooms_list = []
    if not isinstance(rooms, list):
        rooms = [rooms]
    for item in rooms:
        room_datas['hotelId'] = judgment('HotelCode')
        room_datas['roomName'] = judgment(['RoomTypeName'], data=item)
        room_datas['roomTypeCode'] = judgment(
            ['TypeRoom', 'RoomTypeCode'], data=item)
        room_datas['standardOccupancy'] = judgment(
            ['TypeRoom', 'StandardOccupancy'], data=item)
        room_datas['nameSort'] = judgment(['TypeRoom', 'Name'], data=item)
        room_datas['floor'] = judgment(['TypeRoom', 'Floor'], data=item)
        room_datas['invBlockCode'] = judgment(
            ['TypeRoom', 'InvBlockCode'], data=item)
        room_datas['nonSmoking'] = judgment(
            ['TypeRoom', 'NonSmoking'], data=item)
        room_datas['hasWindow'] = judgment(
            ['TypeRoom', 'HasWindow'], data=item)
        room_datas['quantity'] = judgment(['TypeRoom', 'Quantity'], data=item)
        room_datas['roomSize'] = judgment(['TypeRoom', 'RoomSize'], data=item)
        room_datas['bedSize'] = judgment(['TypeRoom', 'Size'], data=item)
        room_datas['bedType'] = judgment(
            ['TypeRoom', 'BedTypeCode'], data=item)
        room_datas['HID'] = judgment(['TypeRoom', 'HID'], data=item)
        if judgment('Amenities', data=item):
            room_datas['amentities'] = judgment(
                ['Amenities', 'Amenity'], data=item)
        room_datas['tpa_Extensions'] = clean_datas['tpa_Extensions']
        rooms_list.append(room_datas)
    clean_datas['rooms'] = rooms_list
    # rooms 字段为rooms_list 完成
#     self.mdb['jiudian_hotels'].update_one({'id': clean_datas['id']},
#                                           {'$set': clean_datas},
#                                           upsert=True)
# #     hotel_price.delay(hotel_id)  # async
#     pprint(clean_datas)
#     pprint(xmlt_data)
#     return [clean_datas['id'], 'message is over']


In [444]:
data=hotel_message_fetch(793625)['Response']['HotelResponse']['OTA_HotelDescriptiveInfoRS']['HotelDescriptiveContents']['HotelDescriptiveContent']
data[1].keys()

dict_keys(['AreaID', 'TPA_Extensions', 'Policies', 'BrandCode', 'ContactInfos', 'HotelCityCode', 'MultimediaDescriptions', 'AffiliationInfo', 'HotelCode', 'FacilityInfo', 'HotelInfo', 'HotelName', 'AreaInfo'])

In [462]:
data[1]

{'AffiliationInfo': {'Awards': {'Award': [{'Provider': 'HotelStarLicence',
     'Rating': '0'},
    {'Provider': 'HotelStarRate', 'Rating': '5'},
    {'Provider': 'CtripStarRate', 'Rating': '2'},
    {'Provider': 'CtripUserRate', 'Rating': '4.5'},
    {'Provider': 'CtripCommRate', 'Rating': '4.6'},
    {'Provider': 'CommSurroundingRate', 'Rating': '4.7'},
    {'Provider': 'CommFacilityRate', 'Rating': '4.4'},
    {'Provider': 'CommCleanRate', 'Rating': '4.6'},
    {'Provider': 'CommServiceRate', 'Rating': '4.6'}]}},
 'AreaID': '181',
 'AreaInfo': {'RefPoints': {'RefPoint': [{'CategoryCode': '1001',
     'DescriptiveText': '中国农业银行',
     'Distance': '0',
     'Latitude': '0',
     'Longitude': '0',
     'Name': '中国农业银行',
     'RefPointCategoryCode': '3',
     'RefPointName': '娱乐',
     'UnitOfMeasureCode': '2'},
    {'CategoryCode': '1001',
     'DescriptiveText': '风花雪月卡拉ok',
     'Distance': '0.822324145334486',
     'Latitude': '32.04002',
     'Longitude': '118.7847',
     'Name': '风

In [551]:
data[1]['FacilityInfo']['GuestRooms']['GuestRoom'][0]['TPA_Extensions']['TPA_Extension']

[{'FTypeName': '媒体/科技', 'FacilityName': '国内长途电话', 'FacilityValue': '1'},
 {'FTypeName': '媒体/科技', 'FacilityName': '国际长途电话', 'FacilityValue': '1'},
 {'FTypeName': '浴室', 'FacilityName': '拖鞋', 'FacilityValue': '1'},
 {'FTypeName': '便利设施', 'FacilityName': '雨伞', 'FacilityValue': '1'},
 {'FTypeName': '便利设施', 'FacilityName': '书桌', 'FacilityValue': '1'},
 {'FTypeName': '浴室', 'FacilityName': '浴室化妆放大镜', 'FacilityValue': '1'},
 {'FTypeName': '浴室', 'FacilityName': '24小时热水', 'FacilityValue': '1'},
 {'FTypeName': '食品和饮品', 'FacilityName': '电热水壶', 'FacilityValue': '1'},
 {'FTypeName': '食品和饮品', 'FacilityName': '咖啡壶/茶壶', 'FacilityValue': '1'},
 {'FTypeName': '浴室', 'FacilityName': '免费洗漱用品(6样以上)', 'FacilityValue': '1'},
 {'FTypeName': '食品和饮品', 'FacilityName': '免费瓶装水', 'FacilityValue': '1'},
 {'FTypeName': '食品和饮品', 'FacilityName': '迷你吧', 'FacilityValue': '1'},
 {'FTypeName': '便利设施', 'FacilityName': '熨衣设备', 'FacilityValue': '1'},
 {'FTypeName': '食品和饮品', 'FacilityName': '小冰箱', 'FacilityValue': '1'},
 {'FTypeN

In [449]:
data[1]['FacilityInfo']['GuestRooms']['GuestRoom'][0]

{'Amenities': {'Amenity': [{'DescriptiveText': '免费有线宽带',
    'RoomAmenityCode': '3'},
   {'DescriptiveText': '免费无线宽带', 'RoomAmenityCode': '5'},
   {'DescriptiveText': '自助早餐', 'RoomAmenityCode': '10'}]},
 'RoomTypeName': '假日高级间(洲际热卖)',
 'TPA_Extensions': {'TPA_Extension': [{'FTypeName': '媒体/科技',
    'FacilityName': '国内长途电话',
    'FacilityValue': '1'},
   {'FTypeName': '媒体/科技', 'FacilityName': '国际长途电话', 'FacilityValue': '1'},
   {'FTypeName': '浴室', 'FacilityName': '拖鞋', 'FacilityValue': '1'},
   {'FTypeName': '便利设施', 'FacilityName': '雨伞', 'FacilityValue': '1'},
   {'FTypeName': '便利设施', 'FacilityName': '书桌', 'FacilityValue': '1'},
   {'FTypeName': '浴室', 'FacilityName': '浴室化妆放大镜', 'FacilityValue': '1'},
   {'FTypeName': '浴室', 'FacilityName': '24小时热水', 'FacilityValue': '1'},
   {'FTypeName': '食品和饮品', 'FacilityName': '电热水壶', 'FacilityValue': '1'},
   {'FTypeName': '食品和饮品', 'FacilityName': '咖啡壶/茶壶', 'FacilityValue': '1'},
   {'FTypeName': '浴室', 'FacilityName': '免费洗漱用品(6样以上)', 'FacilityValue':

In [442]:
data[1]['TPA_Extensions']

{'MasterSubHotelIDs': '369782,61451,369784,378831,430509,451336,453407,532004,1537190,2248313,2347224,2366042,2398399,2609608,2775684,2941908,3180211,3295473,3384907,3463264,4384589,4614368,4961053,5319322,5519400,5727132,5772076,6019427,6019681,6057697,6073251,6395866,6425723,6520309,6532629,6651622',
 'Roomquantity': '316',
 'ThemeCategory': {'ThemeCategoryType': [{'Code': '87'},
   {'Code': '121'},
   {'Code': '177'},
   {'Code': '219'},
   {'Code': '19'},
   {'Code': '38'},
   {'Code': '69'},
   {'Code': '70'},
   {'Code': '79'},
   {'Code': '81'}]}}

In [439]:
data[1]['AffiliationInfo']['Awards']['Award']

[{'Provider': 'HotelStarLicence', 'Rating': '0'},
 {'Provider': 'HotelStarRate', 'Rating': '5'},
 {'Provider': 'CtripStarRate', 'Rating': '2'},
 {'Provider': 'CtripUserRate', 'Rating': '4.5'},
 {'Provider': 'CtripCommRate', 'Rating': '4.6'},
 {'Provider': 'CommSurroundingRate', 'Rating': '4.7'},
 {'Provider': 'CommFacilityRate', 'Rating': '4.4'},
 {'Provider': 'CommCleanRate', 'Rating': '4.6'},
 {'Provider': 'CommServiceRate', 'Rating': '4.6'}]

In [435]:
data[1]['AreaInfo']['RefPoints']['RefPoint']

[{'CategoryCode': '1001',
  'DescriptiveText': '中国农业银行',
  'Distance': '0',
  'Latitude': '0',
  'Longitude': '0',
  'Name': '中国农业银行',
  'RefPointCategoryCode': '3',
  'RefPointName': '娱乐',
  'UnitOfMeasureCode': '2'},
 {'CategoryCode': '1001',
  'DescriptiveText': '风花雪月卡拉ok',
  'Distance': '0.822324145334486',
  'Latitude': '32.04002',
  'Longitude': '118.7847',
  'Name': '风花雪月卡拉ok',
  'RefPointCategoryCode': '3',
  'RefPointName': '娱乐',
  'UnitOfMeasureCode': '2'},
 {'CategoryCode': '1001',
  'DescriptiveText': '金榜自助式卡拉ok',
  'Distance': '0.98310731865801',
  'Latitude': '32.02474',
  'Longitude': '118.79022',
  'Name': '金榜自助式卡拉ok',
  'RefPointCategoryCode': '3',
  'RefPointName': '娱乐',
  'UnitOfMeasureCode': '2'},
 {'CategoryCode': '1001',
  'DescriptiveText': '水平方商场',
  'Distance': '0',
  'Latitude': '0',
  'Longitude': '0',
  'Name': '水平方商场',
  'RefPointCategoryCode': '2',
  'RefPointName': '购物',
  'UnitOfMeasureCode': '2'},
 {'CategoryCode': '1001',
  'DescriptiveText': '188购物中心'

In [431]:
data[1]['Policies']['Policy']['PolicyInfoCodes']['PolicyInfoCode']['Description']

[{'Name': 'ArrivalAndDeparture', 'Text': '入住时间：14:00以后      离店时间：12:00以前'},
 {'Name': 'Cancel', 'Text': '不同类型的客房附带不同的取消预订和预先付费政策 选择上述客房时，请参阅“客房政策”'},
 {'Name': 'DepositAndPrepaid',
  'Text': '入住时需要出示政府核发的身份证件(带照片)。请携带信用卡和现金用以支付押金或额外费用。'},
 {'Name': 'Pet', 'Text': '不可携带宠物。'},
 {'Name': 'Requirements', 'Text': '信用卡授权解除需时1-3个月。视不同国家、城市之银行操作时间而定。'},
 {'Name': 'Child', 'Text': '不接受18岁以下客人单独入住。'},
 {'Name': 'Meals', 'Text': '自助早餐 RMB 78.00'}]

In [425]:
data[1]['HotelInfo']['WhenBuilt']

'2008-10-1 0:00:00'

In [423]:
data[1]['HotelInfo']['LastUpdated']

'2008-10-01T00:00:00'

In [420]:
data[1]['HotelInfo']['IsOnlineFranchise']

'false'

In [417]:
data[1]['HotelInfo']['CategoryCodes']['SegmentCategory']

{'Code': '4'}

In [412]:
data[1]['HotelInfo']['Position']['Latitude']

'32.024223'

In [413]:
data[1]['HotelInfo']['Position']['Longitude']

'118.786014'

In [400]:
data[1]['HotelInfo']['Address']

{'AddressLine': '建康路1号',
 'CityName': '南京',
 'PostalCode': '210001',
 'TPA_Extensions': {'RoadCross': {'DescriptionText': '夫子庙斜对面，近中华路'}},
 'Zone': {'ZoneCode': '71', 'ZoneName': '夫子庙地区'}}

In [575]:
data[1]['MultimediaDescriptions']['MultimediaDescription']

[{'ImageItems': {'ImageItem': [{'Category': '1',
     'Description': {'Caption': '外观'},
     'ImageFormat': {'URL': 'http://dimg04.c-ctrip.com/images/hotel/62000/61451/7fdc7c6ff8e04187aab59905cfa22dae_R_550_412.jpg'},
     'TPA_Extensions': {'InvBlockCode': '0'}},
    {'Category': '1',
     'Description': {'Caption': '外观'},
     'ImageFormat': {'URL': 'http://dimg04.c-ctrip.com/images/hotel/62000/61451/35e3489ecfcd4b3f9fbcb5cd6b639c10_R_550_412.jpg'},
     'TPA_Extensions': {'InvBlockCode': '0'}},
    {'Category': '1',
     'Description': {'Caption': '外观'},
     'ImageFormat': {'URL': 'http://dimg04.c-ctrip.com/images/hotel/370000/369782/656231001e0d476eb61c14b0977d7062_R_550_412.jpg'},
     'TPA_Extensions': {'InvBlockCode': '0'}},
    {'Category': '2',
     'Description': {'Caption': '公共区域'},
     'ImageFormat': {'URL': 'http://dimg04.c-ctrip.com/images/hotel/62000/61451/6be3ddc6fcf5467780218392c1eda671_R_550_412.jpg'},
     'TPA_Extensions': {'InvBlockCode': '0'}},
    {'Category': 

In [397]:
data[1]['MultimediaDescriptions']['MultimediaDescription'][1]['TextItems']['TextItem']

[{'Category': '5',
  'Description': '南京水游城假日酒店，由上海鹏欣（集团）有限公司投资建造、英国洲际酒店集团管理公司进行管理，位于新街口与夫子庙商业圈交汇处，是南京第一家主题购物公园“水游城”的重要组成部分。酒店毗邻三山街地铁1号线，距南京火车站及南京南站仅15分钟地铁车程，并可通过地铁1号线转乘通往机场的地铁直达快线。同时，从酒店至南京禄口国际机场仅35分钟车程。<br><br>酒店拥有客房316间高档客房，其中34间套房、4个行政楼层及配套行政酒廊；无烟楼层及无障碍客房，满足了客人特殊需求；32寸液晶显示屏、卫星电视、互联网宽带接口、语音信箱、国际国内长途、小酒吧、英语报纸等标准设施；WIFI无线网络覆盖所有客房及酒店公共区域。<br><br>假日酒店闻名于世的会议体验—友善的服务、卓越的性价比，满足您多层次的会议需求。面积640平方米，挑高6米的无柱式的宴会厅和五个面积从120平方米到60平米的多功能厅；完善先进的会议设施设备：视听、移动舞台、高速宽带网络和多媒体投影仪等先进设备及专业的会议经理和训练有素的服务队伍确保各类大型招待会、企业论坛、签约仪式、新闻会发布等活动的成功举行。同时也是举办各类宴请、庆典、婚宴等的理想之地。<br><br>酒店西餐厅位于酒店五楼，提供中西合璧的自助餐。提供各种明档现做新鲜、营养的自助菜肴并定期举办各种美食节。餐厅的户外花园定期会举办BBQ烧烤，让忙碌了一天的客人可以在轻松惬意的环境下享受丰盛的自助餐。<br>'},
 {'Category': '11',
  'Description': '南京水游城假日酒店位于新街口与夫子庙商业圈交汇处，毗邻地铁一号线三山街站，交通便利，是南京首家Shopping Park“水游城”的重要组成部分。'},
 {'Category': '501',
  'URL': 'http://u.ctrip.com/union/CtripRedirect.aspx?TypeID=2&Allianceid=328727&sid=810129&OUID=&jumpUrl=http://hotels.ctrip.com/hotel/369782.html'}]

In [395]:
data[1]['ContactInfos']['ContactInfo']['Phones']['Phone']

[{'PhoneNumber': '025-82233888', 'PhoneTechType': '1'},
 {'PhoneNumber': '', 'PhoneTechType': '3'}]

In [393]:
data[1]['MultimediaDescriptions']['MultimediaDescription'][0]['ImageItems']['ImageItem']

[{'Category': '1',
  'Description': {'Caption': '外观'},
  'ImageFormat': {'URL': 'http://dimg04.c-ctrip.com/images/hotel/62000/61451/7fdc7c6ff8e04187aab59905cfa22dae_R_550_412.jpg'},
  'TPA_Extensions': {'InvBlockCode': '0'}},
 {'Category': '1',
  'Description': {'Caption': '外观'},
  'ImageFormat': {'URL': 'http://dimg04.c-ctrip.com/images/hotel/62000/61451/35e3489ecfcd4b3f9fbcb5cd6b639c10_R_550_412.jpg'},
  'TPA_Extensions': {'InvBlockCode': '0'}},
 {'Category': '1',
  'Description': {'Caption': '外观'},
  'ImageFormat': {'URL': 'http://dimg04.c-ctrip.com/images/hotel/370000/369782/656231001e0d476eb61c14b0977d7062_R_550_412.jpg'},
  'TPA_Extensions': {'InvBlockCode': '0'}},
 {'Category': '2',
  'Description': {'Caption': '公共区域'},
  'ImageFormat': {'URL': 'http://dimg04.c-ctrip.com/images/hotel/62000/61451/6be3ddc6fcf5467780218392c1eda671_R_550_412.jpg'},
  'TPA_Extensions': {'InvBlockCode': '0'}},
 {'Category': '2',
  'Description': {'Caption': '公共区域'},
  'ImageFormat': {'URL': 'http://di

In [381]:
data[1]['TPA_Extensions']['ThemeCategory']['ThemeCategoryType']

[{'Code': '87'},
 {'Code': '121'},
 {'Code': '177'},
 {'Code': '219'},
 {'Code': '19'},
 {'Code': '38'},
 {'Code': '69'},
 {'Code': '70'},
 {'Code': '79'},
 {'Code': '81'}]

In [377]:
data[1]['HotelInfo']['Services']['Service']

[{'Code': '1', 'DescriptiveText': '中餐厅', 'ID': '1002'},
 {'Code': '6', 'DescriptiveText': '会议厅', 'ID': '1001'},
 {'Code': '8', 'DescriptiveText': '商务中心', 'ID': '1001'},
 {'Code': '11', 'DescriptiveText': '外币兑换服务', 'ID': '1001'},
 {'Code': '12', 'DescriptiveText': '旅游票务服务', 'ID': '1001'},
 {'Code': '13', 'DescriptiveText': '国内长途电话', 'ID': '1004'},
 {'Code': '14', 'DescriptiveText': '国际长途电话', 'ID': '1004'},
 {'Code': '15', 'DescriptiveText': '洗衣服务', 'ID': '1001'},
 {'Code': '16', 'DescriptiveText': '送餐服务', 'ID': '1001'},
 {'Code': '19', 'DescriptiveText': '残疾人客房', 'ID': '1002'},
 {'Code': '26', 'DescriptiveText': '桌球室', 'ID': '1003'},
 {'Code': '29', 'DescriptiveText': '室内游泳池', 'ID': '1003'},
 {'Code': '42', 'DescriptiveText': '健身室', 'ID': '1003'},
 {'Code': '55', 'DescriptiveText': '叫车服务', 'ID': '1001'},
 {'Code': '69', 'DescriptiveText': '邮政服务', 'ID': '1001'},
 {'Code': '74', 'DescriptiveText': '水上运动', 'ID': '1001'},
 {'Code': '75', 'DescriptiveText': '拖鞋', 'ID': '1004'},
 {'Code': '76

In [165]:
def hotel_price(hotel_id):
    
    start = str(datetime.date.today())
    end=str(datetime.date.today() + datetime.timedelta(days=28))
    ping_url = 'http://{}/Hotel/OTA_HotelRatePlan.asmx'.format(API_URL)
    ts = int(time.time())
    secret = hashlib.md5(KEY.encode('utf8')).hexdigest().upper()
    sign = hashlib.md5('{0}{1}{2}{3}OTA_HotelRatePlan'.format(ts, AID, secret, SID).encode('utf-8')).hexdigest().upper()

    request_xml = ('<?xml version="1.0" encoding="utf-8"?>'
    '<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">'
        '<soap:Body>'
            '<Request xmlns="http://ctrip.com/">'
                '<requestXML><![CDATA['
                    '<?xml version="1.0" encoding="utf-8"?>'
                    '<Request>'
                        '<Header AllianceID="{0}" SID="{1}" TimeStamp="{2}" RequestType="OTA_HotelRatePlan" Signature="{3}"/>'
                        '<HotelRequest>'
                            '<RequestBody xmlns:ns="http://www.opentravel.org/OTA/2003/05" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema">'
                                '<ns:OTA_HotelRatePlanRQ TimeStamp="{7}" Version="1.0">'
                                    '<ns:RatePlans>'
                                        '<ns:RatePlan>'
                                            '<ns:DateRange Start="{4}" End="{5}"/>'
                                            '<ns:RatePlanCandidates>'
                                                '<ns:RatePlanCandidate AvailRatesOnlyInd="true" IsCNYCurrency="true" RatePlanCode=" " >'
                                                    '<ns:HotelRefs>'
                                                        '<ns:HotelRef HotelCode="{6}"/>'
                                                    '</ns:HotelRefs>'
                                                '</ns:RatePlanCandidate>'
                                            '</ns:RatePlanCandidates>'
                                            '<ns:TPA_Extensions RestrictedDisplayIndicator="false" />'
                                        '</ns:RatePlan>'
                                    '</ns:RatePlans>'
                                '</ns:OTA_HotelRatePlanRQ>'
                            '</RequestBody>'
                        '</HotelRequest>'
                    '</Request>]]>'
                '</requestXML>'
            '</Request>'
        '</soap:Body>'
    '</soap:Envelope>').format(AID,SID,ts,sign,start,end,hotel_id,datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S.%f+08:00'))
    encode_xml = request_xml.encode('utf-8')
    headers = {'Content-Type': 'text/xml; charset=utf-8', 'Host': 'openapi.ctrip.com', 'Content-Length': len(encode_xml)}
    resp = requests.post(url=ping_url, data=encode_xml, headers=headers).text
    xmlt_data=xmltodict.parse(xmltodict.parse(resp)['soap:Envelope']['soap:Body']['RequestResponse']['RequestResult'])
    #xml 转 dict 
    xmlt_data=(json.loads((json.dumps(xmlt_data,ensure_ascii=False,indent=2)).replace('@','')))
    print(xmlt_data)
    def judgment(keys, data=xmlt_data, default=[]):
        try:
            if isinstance(keys, str)and isinstance(data,dict):
#                 print(data.keys(),keys)
                return data.get(keys, default)
            if isinstance(keys, list)and isinstance(data,dict):
                if not len(keys) == 1:
                    if judgment(keys[0], data=data):
                        return judgment(keys[1:], data=judgment(keys[0], data=data))
                else:
                    return judgment(keys[-1], data=data)
            else :
                return []
        except KeyError as e:
            return []
        
    items=judgment(['Response','HotelResponse','OTA_HotelRatePlanRS','RatePlans','RatePlan'])
    print(judgment(['Response','HotelResponse','OTA_HotelRatePlanRS','RatePlans','HotelCode']))
    def price_fetch(items):
        prices=[]
        if not items:
            return -1
        else:
            for item in items:
                item=judgment(['Rates','Rate'],data=item)
                if not item:
                    return -1
                else:
                    for ite in item:
                        
                        price=judgment(['BaseByGuestAmts','BaseByGuestAmt','AmountBeforeTax'],data=ite,default=-1)
                        
                        try:
                            prices.append(int(float(price)))
#                             print(json.dumps(int(float(price))))
                        except Exception as e:
                            pass
        return min(prices)
    price=price_fetch(items)
#     hotel.update_one({'id':str(hotel_id)},{'$set':{'price':price}})
    print(hotel_id,price)

In [169]:
hotel_price(1228204)

{'Response': {'Header': {'AccessCount': '100', 'ResultCode': 'Success', 'ShouldRecordPerformanceTime': 'False', 'ResetTime': '2017-01-12 15:27:19', 'ReferenceID': '694b8c32-efbc-4972-8226-b45646651640', 'CurrentCount': '4', 'RecentlyTime': '2017-01-12 15:26:19', 'Timestamp': '2017-01-12T15:26:19.6309003+08:00'}, 'HotelResponse': {'OTA_HotelRatePlanRS': {'Success': None, 'xmlns': 'http://www.opentravel.org/OTA/2003/05', 'Version': '1.0', 'RatePlans': {'HotelCode': '1228204'}, 'PrimaryLangID': 'zh', 'TimeStamp': '2017-01-12T15:26:19.8639293+08:00'}}}}
1228204
1228204 -1


In [321]:
mm[0]['Rates']['Rate'][0]['BaseByGuestAmts']['BaseByGuestAmt']['AmountBeforeTax']

'888.00'

In [312]:
items[0]['Rates']['Rate'][0]['BaseByGuestAmts']['BaseByGuestAmt']['AmountBeforeTax']

'288.00'

In [244]:
datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S.%f+08:00')

'2017-01-03T16:56:55.183386+08:00'

In [634]:
end = datetime.datetime.now()- datetime.timedelta(minutes=1)
end=end.strftime("%Y-%m-%dT%H:%M:%S")
start = datetime.datetime.now() - datetime.timedelta(hours=6)
start=start.strftime("%Y-%m-%dT%H:%M:%S")
print(start)
print(end)

2017-01-04T04:03:22
2017-01-04T10:02:22


## 酒店静态信息增量接口

In [163]:
def get_hotel_info_increment(city_id):
    ping_url = 'http://{}/hotel/GetHotelInfoIncrement.asmx'.format(API_URL)
    page_index=1
    i=True #while 循环控制
    while i :
        ts = int(time.time())
        end = datetime.datetime.now()- datetime.timedelta(minutes=10)
        end=end.strftime("%Y-%m-%dT%H:%M:%S")
        start = datetime.datetime.now() - datetime.timedelta(hours=5)
        start=start.strftime("%Y-%m-%dT%H:%M:%S")

        secret = hashlib.md5(KEY.encode('utf8')).hexdigest().upper()
        sign = hashlib.md5('{0}{1}{2}{3}GetHotelInfoIncrement'.format(
            ts, AID, secret, SID).encode('utf-8')).hexdigest().upper()

        request_xml = ('<?xml version="1.0" encoding="utf-8"?>'
                       '<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">'
                       '<soap:Body>'
                       '<Request xmlns="http://ctrip.com/">'
                       '<requestXML><![CDATA['
                       '<?xml version="1.0" encoding="utf-8"?>'
                       '<Request>'
                       '<Header AllianceID="{0}" SID="{1}" TimeStamp="{2}" RequestType="GetHotelInfoIncrement" Signature="{3}" />'
                        '<GetHotelInfoIncrementRequest>'
                        '<StartTime>{4}</StartTime>'
                        '<EndTime>{5}</EndTime>'
                        '<CityID>{6}</CityID>'
                        '<PageSize>10</PageSize>'
                        '<PageIndex>{7}</PageIndex>'
                        '<CountryID>1</CountryID>'
                        '</GetHotelInfoIncrementRequest>'
                       '</Request>]]>'
                       '</requestXML>'
                       '</Request>'
                       '</soap:Body>'
                       '</soap:Envelope>').format(AID, SID, ts, sign,start,end,city_id,page_index)

        encode_xml = request_xml.encode('utf-8')

        headers = {'Content-Type': 'text/xml; charset=utf-8',
                   'Host': 'openapi.ctrip.com', 'Content-Length': len(encode_xml)}
        resp = requests.post(url=ping_url, data=encode_xml, headers=headers).text

        xmlt_data = xmltodict.parse(xmltodict.parse(resp)['soap:Envelope'][
                                   'soap:Body']['RequestResponse']['RequestResult'])
        # xml 转 dict
        xmlt_data = (json.loads(
            (json.dumps(xmlt_data, ensure_ascii=False, indent=2)).replace('@', '')))
        # 代码健壮性之首要！
        def judgment(keys, data=xmlt_data, default=[]):
            try:
                if isinstance(keys, str)and isinstance(data,dict):
                    return data.get(keys, default)
                if isinstance(keys, list)and isinstance(data,dict):
                    if not len(keys) == 1:
                        if judgment(keys[0], data=data):
                            return judgment(keys[1:], data=judgment(keys[0], data=data))
                    else:
                        return judgment(keys[-1], data=data)
                else :
                    return []
            except KeyError as e:
                return []
        access_count=judgment(['Response','Header','AccessCount'])
        hotel_ids=judgment(['Response','GetHotelInfoIncrementResponse','HotelInfoChangeList','HotelID'])
        if hotel_ids:
            if isinstance(hotel_ids,str):
                hotel_ids=[hotel_ids]
            page_index+=1
            for hotel_id in hotel_ids:
                hotel_message_fetch(hotel_id)#async
        else:
            i=False
    return ['city',city,'update increment is over']

In [164]:
get_hotel_info_increment(10)

{
  "Response": {
    "GetHotelInfoIncrementResponse": {
      "HotelInfoChangeList": {
        "HotelID": [
          "469944",
          "434294",
          "5266764",
          "1712401",
          "2352601",
          "5615267",
          "606767",
          "439885",
          "5293482",
          "430213",
          "2361830",
          "6207560",
          "1635598",
          "2152165",
          "2496791",
          "1947649",
          "1338946",
          "2630948",
          "2711360",
          "2058885"
        ]
      }
    },
    "Header": {
      "AccessCount": "5000",
      "ResultCode": "Success",
      "ShouldRecordPerformanceTime": "False",
      "ResetTime": "2017-01-12 10:45:08",
      "ReferenceID": "2b3c8975-05f3-42fd-bcbe-712c229861e8",
      "CurrentCount": "37",
      "RecentlyTime": "2017-01-12 10:44:08",
      "Timestamp": "2017-01-12T10:44:10.6803677+08:00"
    }
  }
}
{
  "Response": {
    "GetHotelInfoIncrementResponse": {
      "HotelInfoChangeList": {

NameError: name 'city' is not defined

## 通过酒店ID全量列表更新酒店数据 pass

In [29]:
def get_hotel_info_increment(city_id, page_index=1):
    """查询city所属酒店增量ID信息，并调用静态信息查询接口"""
    ping_url = 'http://{}/hotel/GetHotelInfoIncrement.asmx'.format(API_URL)
    ts = int(time.time())
    end = datetime.datetime.now() - datetime.timedelta(minutes=10)
    end = end.strftime("%Y-%m-%dT%H:%M:%S")
    start = datetime.datetime.now() - datetime.timedelta(hours=5)
    start = start.strftime("%Y-%m-%dT%H:%M:%S")

    secret = hashlib.md5(KEY.encode('utf8')).hexdigest().upper()
    sign = hashlib.md5('{0}{1}{2}{3}GetHotelInfoIncrement'.format(
        ts, AID, secret, SID).encode('utf-8')).hexdigest().upper()

    request_xml = ('<?xml version="1.0" encoding="utf-8"?>'
                    '<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">'
                    '<soap:Body>'
                    '<Request xmlns="http://ctrip.com/">'
                    '<requestXML><![CDATA['
                    '<?xml version="1.0" encoding="utf-8"?>'
                    '<Request>'
                    '<Header AllianceID="{0}" SID="{1}" TimeStamp="{2}" RequestType="GetHotelInfoIncrement" Signature="{3}" />'
                    '<GetHotelInfoIncrementRequest>'
                    '<StartTime>{4}</StartTime>'
                    '<EndTime>{5}</EndTime>'
                    '<CityID>{6}</CityID>'
                    '<PageSize>10</PageSize>'
                    '<PageIndex>{7}</PageIndex>'
                    '<CountryID>1</CountryID>'
                    '</GetHotelInfoIncrementRequest>'
                    '</Request>]]>'
                    '</requestXML>'
                    '</Request>'
                    '</soap:Body>'
                    '</soap:Envelope>').format(AID, SID, ts, sign, start, end, city_id, page_index)

    encode_xml = request_xml.encode('utf-8')

    headers = {
        'Content-Type': 'text/xml; charset=utf-8',
        'Host': 'openapi.ctrip.com',
        'Content-Length': len(encode_xml)
    }
    resp = requests.post(url=ping_url, data=encode_xml, headers=headers).text
    xmlt_data = xmltodict.parse(xmltodict.parse(resp)['soap:Envelope'][
        'soap:Body']['RequestResponse']['RequestResult'])
    # xml 转 dict
    xmlt_data = (json.loads(
        (json.dumps(xmlt_data, ensure_ascii=False, indent=2)).replace('@', '')))

    # 代码健壮性之首要！
    def judgment(keys, data=xmlt_data, default=[]):
        try:
            if isinstance(keys, str)and isinstance(data, dict):
                return data.get(keys, default)
            if isinstance(keys, list)and isinstance(data, dict):
                if not len(keys) == 1:
                    if judgment(keys[0], data=data):
                        return judgment(keys[1:], data=judgment(keys[0], data=data))
                else:
                    return judgment(keys[-1], data=data)
            else:
                return []
        except KeyError as e:
            return []
    access_count = judgment(['Response', 'Header', 'AccessCount'])
    hotel_ids = judgment(
        ['Response', 'GetHotelInfoIncrementResponse', 'HotelInfoChangeList', 'HotelID'])
    if hotel_ids:
        if isinstance(hotel_ids, str):
            hotel_ids = [hotel_ids]
#         for hotel_id in hotel_ids:
#             hotel_message_fetch.delay(hotel_id)  # async
        get_hotel_info_increment(city_id,page_index+1)
        print(city_id,page_index,'over')
    else:
        print(city_id,'is OK')
#     return ['city', city_id, 'update increment is over']


In [1]:
def update_hotel():
    """更新上下线酒店增量信息"""
    with open(r"/home/fanjindong/notebook/celery/celery_demo/celery_app/携程国内城市.xml") as f:
        data = f.read()
    data_json = xmltodict.parse(data)
    items = data_json['CityDetails']['CityDetail']
    for item in items:
        city = int(item['City'])
        get_hotel_info_increment.delay(city)  # async
    return ['OVER!']

In [30]:
get_hotel_info_increment(7)

7 is OK
7 15 over
7 14 over
7 13 over
7 12 over
7 11 over
7 10 over
7 9 over
7 8 over
7 7 over
7 6 over
7 5 over
7 4 over
7 3 over
7 2 over
7 1 over


In [130]:
with open(r"/home/fanjindong/notebook/celery/celery_demo/celery_app/携程国内城市.xml") as f:
    data = f.read()
data_json = xmltodict.parse(data)
items = data_json['CityDetails']['CityDetail']
# print(items[0])
citys=set()
for item in items:
    citys.add(item['CityName'])
print(citys,len(list(citys)))

{'塔源', '怀化', '安顺', '通辽北', '赫章', '波密', '双涟', '于都', '嘉黎', '盐津', '长顺', '周村', '嘉峰', '马踪滩', '长沙坝', '清原', '海阳', '南王', '东坝', '大成', '鸣李', '阳城', '常宁', '二道沟', '商河', '天峨', '捏掌', '盂县', '石门子', '定日', '邓州', '宝泉岭', '东港', '遂宁', '聂村', '蔚州', '墙子路', '崇阳', '黔城', '佛坪', '四道湾', '华蓥', '营盘水', '祁连', '灵武', '师宗', '上交', '黄丝', '连南', '上饶县', '井冈山', '资中', '河市坝', '中牟', '临桂', '绿化', '朱石寨', '拉林', '庄浪', '封开', '苏维', '平台', '巫山', '丹寨', '寿阳', '清徐', '长胜', '兴业', '蔡家', '翠岗', '台东', '南定', '兴南', '琼中', '玉屏', '布拖', '南塘', '杭锦后旗', '陀山', '黎川', '绍兴县', '上虞', '南沟', '五十家子', '大理', '张贵庄', '云霄', '渠县', '靖宇', '通辽', '三道沟', '昂昂溪', '孝泉', '墨江', '庆安', '淳化', '干塘', '腰栈', '陇川', '隆林', '垫江', '楠溪江', '宁阳县', '琼山', '乌兰浩特', '雷山', '和布克赛尔', '新兴隆', '丰台', '芒康', '平昌', '小金', '张辛', '当雄', '樟树', '静乐', '宿松', '望谟', '南海市', '明溪', '民权', '泰山', '望溪', '金城江', '瓢儿屯', '萨拉齐', '田林', '平安', '顺义', '东乡族', '苏州', '安福', '临猗', '平山', '于家庄', '贵港', '古固寨', '孔家', '建阳', '东元庆', '磐安', '台前', '山阳', '乌海', '永德', '墨冲', '赵县', '嘎什甸子', '徘徊北', '石景山', '牡丹江', '沂水', '平旺', '南白镇', '玉门', '五道木', '碾子山', '蒲圻', '公主岭',

### 历史版本

In [ ]:
    def judgment(keys, data=dirty_datas, default=[]):
        """传入keys=[key1,key2,key3],
           执行data[key1][key2][key3],
           防止报错，若获取失败则返回[] or None
        """
        try:
            if isinstance(keys, str):
                return data.get(keys, default)
            if isinstance(keys, list):
                if not len(keys) == 1:
                    if judgment(keys[0], data=data):
                        return judgment(keys[1:], data=judgment(keys[0], data=data))
                else:
                    return judgment(keys[-1], data=data)
        except KeyError as e:
            return []

### judgment函数v0.2

In [110]:
def judgment(keys, data=None, default=None):
    """传入keys=[key1,key2,key3],
       执行data[key1][key2][key3],
       防止报错，若获取失败则返回[] or None
    """
    if data is None:
        data={'1':1,'2':{'0':{'3':203}}}
    if default is None:
        default=[]
    try:
        if isinstance(keys, str):
            return data.get(keys, default)
        if isinstance(keys, list):
            if len(keys)==1:
                return data.get(keys[0],default)
            else:
                if data.get(keys[0],default):
                    return judgment(keys[1:], data=data.get(keys[0],default))
                else:
                    return default
    except (KeyError,AttributeError) as e:
        return default
judgment(['2'])

{'0': {'3': 203}}

## judgment函数v0.3

In [369]:
def judgment(keys, data=None, default=None):
    """传入keys=[key1,key2,key3],
       执行data[key1][key2][key3],
       防止报错，若获取失败则返回[] or None
    """
    if data is None:
        data={'1':{1},'2':{'0':{'3':203}}}
    if default is None:
        default=[]
    try:
        if isinstance(keys, str):
            return data[keys]
        elif isinstance(keys, list):    
            if len(keys)==1:
                return data[keys[0]]
            else:
                return judgment(keys[1:], data=data[keys[0]])
        else:
            return default
    except (KeyError,AttributeError,TypeError) as e:
        return default
judgment(['1','2'])

[]

## 酒店静态信息接口
+ 实现一次接口请求，传入至多10个酒店id，返回至多10个酒店信息

In [552]:
def data_purge(dirty_datas):
    """数据清洗、插入数据库
    """
    # dirty_datas = {}  # 存放爬取接口所返回的初始数据
    clean_datas = {}  # 存放经映射后的静态信息数据
    room_datas = {}  # 存放经映射后的酒店房型数据集
    def judgment(keys, data=None, default=None):
        """ 传入keys=[key1,key2,key3],
            执行data[key1][key2][key3],
            防止报错，若获取失败则返回[] or None
        """
        if data is None:
            data = dirty_datas
        try:
            if isinstance(keys, str):
                return data[keys]
            elif isinstance(keys, list):
                if len(keys) == 1:
                    return data[keys[0]]
                else:
                    return judgment(keys[1:], data=data[keys[0]])
            else:
                return default
        except (KeyError, AttributeError, TypeError) as e:
            return default
    # 映射数据结构
    clean_datas['id'] = judgment('HotelCode')
    clean_datas['name'] = judgment('HotelName')
    clean_datas['brandCode'] = judgment('BrandCode')
    clean_datas['cityCode'] = judgment('HotelCityCode')
    clean_datas['areaId'] = judgment('AreaID')

    items = judgment(['HotelInfo', 'Services', 'Service'])
    if not isinstance(items, list):
        items = [items]
    clean_datas['services'] = []
    for item in items:
        clean_datas['services'].append(judgment('Code', data=item))

    items = judgment(['TPA_Extensions', 'ThemeCategory', 'ThemeCategoryType'])
    if not isinstance(items, list):
        items = [items]
    clean_datas['themes'] = []
    for item in items:
        if judgment('Code', data=item):
            clean_datas['themes'].append(judgment('Code', data=item))
    # 'images','descriptions'字段在一个list中，故放在一起处理
    clean_datas['images'] = []
    clean_datas['descriptions'] = []
    items_desc = judgment(['MultimediaDescriptions', 'MultimediaDescription'])
#     pprint(items_desc)
    items = judgment(['ImageItems', 'ImageItem'], data=items_desc[0])
    if not isinstance(items, list):
        items = [items]
    for item in items:
        clean_datas['images'].append({
            'category': judgment('Category', data=item),
            'caption': judgment(["Description", 'Caption'], data=item),
            'url': judgment(["ImageFormat", 'URL'], data=item)
        })

    items = judgment(['TextItems', 'TextItem'], data=items_desc[1])
    if not isinstance(items, list):
        items = [items]
    for item in items:
        if item.get('Category') and item.get('Description'):
            clean_datas['descriptions'].append(
                {'category': item['Category'], 'content': item['Description']})

    clean_datas['phones'] = []
    items = judgment(['ContactInfos', 'ContactInfo', 'Phones', 'Phone'])
    if not isinstance(items, list):
        items = [items]
    for item in items:
        if judgment('PhoneNumber', data=item):
            clean_datas['phones'].append(item['PhoneNumber'])

    clean_datas['address'] = judgment(['HotelInfo', 'Address'])
    clean_datas['lat'] = judgment(['HotelInfo', 'Position', 'Latitude'])
    clean_datas['lng'] = judgment(['HotelInfo', 'Position', 'Longitude'])
    clean_datas['loc'] = [clean_datas['lng'], clean_datas['lat']]

    clean_datas['categoryCodes'] = {
        "SegmentCategory" : judgment(
            ['HotelInfo', 'CategoryCodes', 'SegmentCategory', 'Code']
        )
    }
    clean_datas['isOnlineFranchise'] = judgment(
        ['HotelInfo', 'IsOnlineFranchise'])
    clean_datas['lastUpdated'] = judgment(['HotelInfo', 'LastUpdated'])
    if judgment('WhenBuilt'):
        clean_datas['whenBuilt'] = judgment(['HotelInfo', 'WhenBuilt'])
    dl = []
    items = judgment(['Policies', 'Policy', 'PolicyInfoCodes',
                      'PolicyInfoCode', 'Description'])
    if not isinstance(items, list):
        items = [items]
    for item in items:
        if isinstance(item, dict):
            if item.get('Text') and item.get('Name'):
                dl.append({'Text': judgment('Text', item),
                           'Name': judgment('Name', item)})
    clean_datas['policies'] = {
        "PolicyInfo": {
            "CheckOutTime": judgment(
                ['Policies', 'Policy', 'PolicyInfo', 'CheckOutTime']),
            "CheckInTime": judgment(
                ['Policies', 'Policy', 'PolicyInfo', 'CheckInTime'])
        },
        "PolicyInfoCodes": {
            "PolicyInfoCode": {
                "Description": dl,
                "Code": judgment(
                    ['Policies', 'Policy',
                     'PolicyInfoCodes', 'PolicyInfoCode', 'Code'])
            }
        }
    }
    clean_datas['refPoints'] = judgment(['AreaInfo', 'RefPoints', 'RefPoint'])
    clean_datas['awards'] = {}
    items = judgment(['AffiliationInfo', 'Awards', 'Award'])
    if not isinstance(items, list):
        items = [items]
    for item in items:
        if judgment('Provider', data=item):
            clean_datas['awards'][judgment('Provider', data=item)] = judgment(
                'Rating', data=item)
    clean_datas['tpa_Extensions'] = judgment('TPA_Extensions')

    # room
    rooms = judgment(['FacilityInfo', 'GuestRooms', 'GuestRoom'])
    rooms_list = []
    if not isinstance(rooms, list):
        rooms = [rooms]
    for item in rooms:
        room_datas['hotelId'] = judgment('HotelCode')
        room_datas['roomName'] = judgment(['RoomTypeName'], data=item)
        room_datas['roomTypeCode'] = judgment(
            ['TypeRoom', 'RoomTypeCode'], data=item)
        room_datas['standardOccupancy'] = judgment(
            ['TypeRoom', 'StandardOccupancy'], data=item)
        room_datas['nameSort'] = judgment(['TypeRoom', 'Name'], data=item)
        room_datas['floor'] = judgment(['TypeRoom', 'Floor'], data=item)
        room_datas['invBlockCode'] = judgment(
            ['TypeRoom', 'InvBlockCode'], data=item)
        room_datas['nonSmoking'] = judgment(
            ['TypeRoom', 'NonSmoking'], data=item)
        room_datas['hasWindow'] = judgment(
            ['TypeRoom', 'HasWindow'], data=item)
        room_datas['quantity'] = judgment(['TypeRoom', 'Quantity'], data=item)
        room_datas['roomSize'] = judgment(['TypeRoom', 'RoomSize'], data=item)
        room_datas['bedSize'] = judgment(['TypeRoom', 'Size'], data=item)
        room_datas['bedType'] = judgment(
            ['TypeRoom', 'BedTypeCode'], data=item)
        room_datas['HID'] = judgment(['TypeRoom', 'HID'], data=item)
        if judgment('Amenities', data=item):
            room_datas['amentities'] = judgment(
                ['Amenities', 'Amenity'], data=item)
        room_datas['tpa_Extensions'] = judgment(
            ['TPA_Extensions', 'TPA_Extension'], data=item)
        rooms_list.append(room_datas)
    clean_datas['rooms'] = rooms_list  # rooms 字段为rooms_list 完成
    self.mdb['jiudian_hotels'].update_one(
        {'id': clean_datas['id']},{'$set': clean_datas},upsert=True)

In [716]:
def hotel_message_fetch(hotel_ids, try_number=1):
    """酒店静态信息查询"""
    ping_url = 'http://{}/Hotel/OTA_HotelDescriptiveInfo.asmx'.format(API_URL)
    ts = int(time.time())
    secret = hashlib.md5(KEY.encode('utf8')).hexdigest().upper()
    sign = hashlib.md5('{0}{1}{2}{3}OTA_Ping'.format(
        ts, AID, secret, SID).encode('utf-8')).hexdigest().upper()
    request_xml = ('<?xml version="1.0" encoding="utf-8"?>'
                   '<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">'
                   '<soap:Body>'
                   '<Request xmlns="http://ctrip.com/">'
                   '<requestXML><![CDATA['
                   '<?xml version="1.0" encoding="utf-8"?>'
                   '<Request>'
                   '<Header AllianceID="{0}" SID="{1}" TimeStamp="{2}" RequestType="OTA_Ping" Signature="{3}" />'
                   '<HotelRequest>'
                   '<RequestBody xmlns:ns="http://www.opentravel.org/OTA/2003/05" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema">'
                   '<OTA_HotelDescriptiveInfoRQ Version="1.0" xsi:schemaLocation="http://www.opentravel.org/OTA/2003/05OTA_HotelDescriptiveInfoRQ.xsd" xmlns="http://www.opentravel.org/OTA/2003/05" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">'
                   '<HotelDescriptiveInfos>'
                   "{4}"
                   '</HotelDescriptiveInfos>'
                   '</OTA_HotelDescriptiveInfoRQ>'
                   '</RequestBody>'
                   '</HotelRequest>'
                   '</Request>]]>'
                   '</requestXML>'
                   '</Request>'
                   '</soap:Body>'
                   '</soap:Envelope>')
    hotel_desc = ('<HotelDescriptiveInfo HotelCode="{0[index]}" PositionTypeCode="502">'
                '<HotelInfo SendData="true"/>'
                '<FacilityInfo SendGuestRooms="true"/>'
                '<AreaInfo SendAttractions="true" SendRecreations="true"/>'
                '<ContactInfo SendData="true"/>'
                '<MultimediaObjects SendData="true"/>'
                '</HotelDescriptiveInfo>')
    hotel_index=''
    n=len(hotel_ids)
    for i in range(n):
        hotel_index+=hotel_desc.replace('index',str(i))
    hotel_desc=hotel_index.format(hotel_ids)
    request_xml=request_xml.format(AID, SID, ts, sign, hotel_desc)
    # pprint(request_xml)
    encode_xml = request_xml.encode('utf-8')

    headers = {'Content-Type': 'text/xml; charset=utf-8',
               'Host': 'openapi.ctrip.com', 'Content-Length': len(encode_xml)}
    try:
        resp = requests.post(url=ping_url, data=encode_xml, headers=headers).text
        xmlt_data = xmltodict.parse(xmltodict.parse(resp)[
            'soap:Envelope']['soap:Body']['RequestResponse']['RequestResult'])
        # xml 转 dict
        xmlt_data = (json.loads(
            (json.dumps(xmlt_data, ensure_ascii=False, indent=2)).replace('@', '')))
        datas=xmlt_data['Response']['HotelResponse']['OTA_HotelDescriptiveInfoRS']['HotelDescriptiveContents']['HotelDescriptiveContent']
    except KeyError as e:
        logging.exception(e)
        return None
    for data in datas[n-1:]:
        data_purge(data)
    return None
hotel_ids=['346276','346278','346283','367765','369707','369710','369713','369717','369782','369785','371145','375288','378810','378829','396597','396947','397164','404182','419459','419566']
hotel_message_fetch(hotel_ids[:10])

{'address': {'AddressLine': '山西路9号',
             'CityName': '南京',
             'PostalCode': '210009',
             'TPA_Extensions': {'RoadCross': {'DescriptionText': '近湖南路'}},
             'Zone': {'ZoneCode': '181', 'ZoneName': '玄武湖湖南路地区'}},
 'areaId': '188',
 'awards': {'CommCleanRate': '4.7',
            'CommFacilityRate': '4.6',
            'CommServiceRate': '4.6',
            'CommSurroundingRate': '4.6',
            'CtripCommRate': '4.6',
            'CtripStarRate': '5',
            'CtripUserRate': '5',
            'HotelStarLicence': '1',
            'HotelStarRate': '5'},
 'brandCode': '6',
 'categoryCodes': {'SegmentCategory': '4'},
 'cityCode': '12',
 'descriptions': [{'category': '5',
                   'content': '南京索菲特银河大酒店是法国雅高酒店集团旗下的奢华酒店品牌，位于南京市中心，置身于繁华的湖南路商务区，体验精致，现代和优雅，饱览南京美景。设计巧妙地融合着法式风情与中国传统元素，从而使南京索菲特银河大酒店成为了令人印象深刻、美妙绝伦的奢华酒店。<br>酒店拥有352间客房，房间的整体设计糅合法式生活品味与中国特色，包含高级房、豪华房、高级套房、行政套房等多种房型，可满足客人多样化的需求。置身房中，可远眺窗外滚滚东流长江水，更能将巍巍紫金山、青青玄武湖壮美风光尽收眼底。<br>7楼设有2个餐厅：锦厨全日制餐厅

## 价格计划查询接口
+ 实现一次接口请求，传入多个酒店ID，并返回多个酒店的价格计划
+ 计算每个酒店价格计划的价格最小值，插入数据库price字段

In [652]:
def rateplans_to_price(rateplans_data): 
    """传入价格日历列表，计算最低价price字段的值，并插入数据库
    """
    def judgment(keys, data=None, default=None):
        """ 传入keys=[key1,key2,key3],
            执行data[key1][key2][key3],
            防止报错，若获取失败则返回[] or None
        """
        if data is None:
            data = rateplans_data
        try:
            if isinstance(keys, str):
                return data[keys]
            elif isinstance(keys, list):
                if len(keys) == 1:
                    return data[keys[0]]
                else:
                    return judgment(keys[1:], data=data[keys[0]])
            else:
                return default
        except (KeyError, AttributeError, TypeError) as e:
            return default
    hotelid = rateplans_data['HotelCode']
    items = rateplans_data['RatePlan']
    def to_price(items):
        """遍历并拿到最小price
        """
        prices = {}
        if not items:
            return -1
        else:
            for item in items:
                item = judgment(['Rates', 'Rate'], data=item)
                if not item:
                    return -1
                else:

                    for ite in item:
                        amount=judgment(
                            ['BaseByGuestAmts', 'BaseByGuestAmt'],
                            data=ite
                        )
                        if amount:
                            key=(float(amount['AmountBeforeTax']))
                            value=(float(amount['ListPrice']))
                            prices[key]=value

        if prices:
            price=min(list(prices.keys()))
            listprice=prices[price]
            return price,listprice
        else:
            return -1,-1
    price,listprice = to_price(items)
    print(price,listprice)
#     self.mdb['jiudian_hotels'].update_one(
#         {'id': hotel_id},
#         {'$set': {'price': price}},
#         upsert=True
#     )
    return [hotelid, price,listprice ,'price insert OK']

In [653]:
def hotel_price_fetch(hotel_ids):
    """查询酒店ID对应price日历，返回最低价格
    """
    start = str(datetime.date.today())
    end = str(datetime.date.today() + datetime.timedelta(days=28))
    ping_url = 'http://{}/Hotel/OTA_HotelRatePlan.asmx'.format(API_URL)
    ts = int(time.time())
    secret = hashlib.md5(KEY.encode('utf8')).hexdigest().upper()
    sign = hashlib.md5('{0}{1}{2}{3}OTA_HotelRatePlan'.format(
        ts, AID, secret, SID).encode('utf-8')).hexdigest().upper()

    request_xml = ('<?xml version="1.0" encoding="utf-8"?>'
                   '<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">'
                   '<soap:Body>'
                   '<Request xmlns="http://ctrip.com/">'
                   '<requestXML><![CDATA['
                   '<?xml version="1.0" encoding="utf-8"?>'
                   '<Request>'
                   '<Header AllianceID="{0}" SID="{1}" TimeStamp="{2}" RequestType="OTA_HotelRatePlan" Signature="{3}"/>'
                   '<HotelRequest>'
                   '<RequestBody xmlns:ns="http://www.opentravel.org/OTA/2003/05" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema">'
                   '<ns:OTA_HotelRatePlanRQ TimeStamp="{4}" Version="1.0">'
                   '<ns:RatePlans>'
                    "{rateplans}"
                   '</ns:RatePlans>'
                   '</ns:OTA_HotelRatePlanRQ>'
                   '</RequestBody>'
                   '</HotelRequest>'
                   '</Request>]]>'
                   '</requestXML>'
                   '</Request>'
                   '</soap:Body>'
                   '</soap:Envelope>')
    rateplan = ('<ns:RatePlan>'
               '<ns:DateRange Start="{start}" End="{end}"/>'
               '<ns:RatePlanCandidates>'
               '<ns:RatePlanCandidate AvailRatesOnlyInd="true" IsCNYCurrency="true" RatePlanCode=" " >'
               '<ns:HotelRefs>'
               '<ns:HotelRef HotelCode="{hotel_ids[index]}"/>'
               '</ns:HotelRefs>'
               '</ns:RatePlanCandidate>'
               '</ns:RatePlanCandidates>'
               '<ns:TPA_Extensions RestrictedDisplayIndicator="false" />'
               '</ns:RatePlan>')
    rateplans=''
    n=len(hotel_ids)
    for index in range(n):
        rateplans+=rateplan.replace('index',str(index))
    rateplans=rateplans.format(hotel_ids=hotel_ids,start=start,end=end)
    request_xml=request_xml.format(AID, SID, ts, sign, datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S.%f+08:00'),rateplans=rateplans)
#     print(request_xml)
    encode_xml = request_xml.encode('utf-8')
    headers = {'Content-Type': 'text/xml; charset=utf-8',
               'Host': 'openapi.ctrip.com', 'Content-Length': len(encode_xml)}
    resp = requests.post(url=ping_url, data=encode_xml, headers=headers).text
    xmlt_data = xmltodict.parse(xmltodict.parse(resp)[
        'soap:Envelope']['soap:Body']['RequestResponse']['RequestResult'])
    # xml 转 dict
    xmlt_data = (json.loads(
        (json.dumps(xmlt_data, ensure_ascii=False, indent=2)).replace('@', '')))
    datas = xmlt_data['Response']['HotelResponse']['OTA_HotelRatePlanRS']['RatePlans']
    if isinstance(datas,list):
        for data in datas:
            rateplans_to_price(data)
    else:
        rateplans_to_price(datas)
    return ['rateplans fetch is OK']
hotel_ids=['346276','346278','346283','367765','369707','369710','369713','369717','369782','369785','371145','375288','378810','378829','396597','396947','397164','404182','419459','419566']
hotel_price_fetch(hotel_ids[:10])

936.0 1768.0
936.0 1768.0
1014.0 1768.0
1014.0 1768.0
827.0 1768.0
827.0 1768.0
1014.0 1768.0
1014.0 1768.0
1014.0 1768.0
846.0 1768.0
1000.0 2338.0
1000.0 2338.0
1000.0 2338.0
1000.0 2338.0
912.0 2338.0
912.0 2338.0
1000.0 2338.0
1000.0 2338.0
1000.0 2338.0
1079.0 2338.0
739.0 1768.0
739.0 1768.0
800.0 1768.0
800.0 1768.0
694.0 1768.0
694.0 1768.0
800.0 1768.0
800.0 1768.0
800.0 1768.0
668.0 1768.0
837.0 1768.0
837.0 1768.0
907.0 1768.0
907.0 1768.0
827.0 1768.0
827.0 1768.0
907.0 1768.0
907.0 1768.0
907.0 1768.0
757.0 1768.0
739.0 1768.0
739.0 1768.0
800.0 1768.0
800.0 1768.0
694.0 1768.0
694.0 1768.0
800.0 1768.0
800.0 1768.0
800.0 1768.0
668.0 1768.0
1112.0 2218.0
1112.0 2218.0
1112.0 2218.0
1112.0 2218.0
1023.0 2218.0
1023.0 2218.0
1112.0 2218.0
1112.0 2218.0
1112.0 2218.0
1198.0 2218.0
739.0 1768.0
739.0 1768.0
800.0 1768.0
800.0 1768.0
694.0 1768.0
694.0 1768.0
800.0 1768.0
800.0 1768.0
800.0 1768.0
668.0 1768.0
641.0 1758.0
648.0 1758.0
670.0 1758.0
670.0 1758.0
651.0 1758.0
63

['rateplans fetch is OK']

## 酒店静态信息增量接口

In [730]:
def get_hotel_info_increment(city_id, page_index=1):
    """查询city所属酒店增量ID信息，并调用静态信息查询接口"""
    ping_url = 'http://{}/hotel/GetHotelInfoIncrement.asmx'.format(API_URL)
    ts = int(time.time())
    end = datetime.datetime.now() - datetime.timedelta(minutes=1)
    end = end.strftime("%Y-%m-%dT%H:%M:%S")
    start = datetime.datetime.now() - datetime.timedelta(minutes=16)
    start = start.strftime("%Y-%m-%dT%H:%M:%S")

    secret = hashlib.md5(KEY.encode('utf8')).hexdigest().upper()
    sign = hashlib.md5('{0}{1}{2}{3}GetHotelInfoIncrement'.format(
        ts, AID, secret, SID).encode('utf-8')).hexdigest().upper()

    request_xml = ('<?xml version="1.0" encoding="utf-8"?>'
                   '<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">'
                   '<soap:Body>'
                   '<Request xmlns="http://ctrip.com/">'
                   '<requestXML><![CDATA['
                   '<?xml version="1.0" encoding="utf-8"?>'
                   '<Request>'
                   '<Header AllianceID="{0}" SID="{1}" TimeStamp="{2}" RequestType="GetHotelInfoIncrement" Signature="{3}" />'
                   '<GetHotelInfoIncrementRequest>'
                   '<StartTime>{4}</StartTime>'
                   '<EndTime>{5}</EndTime>'
                   '<CityID>{6}</CityID>'
#                    '<PageSize>10</PageSize>'
                   '<PageIndex>{7}</PageIndex>'
                   '<CountryID>1</CountryID>'
                   '</GetHotelInfoIncrementRequest>'
                   '</Request>]]>'
                   '</requestXML>'
                   '</Request>'
                   '</soap:Body>'
                   '</soap:Envelope>').format(AID, SID, ts, sign, start, end, city_id, page_index)

    encode_xml = request_xml.encode('utf-8')

    headers = {
        'Content-Type': 'text/xml; charset=utf-8',
        'Host': 'openapi.ctrip.com',
        'Content-Length': len(encode_xml)
    }
    resp = requests.post(url=ping_url, data=encode_xml, headers=headers).text
    xmlt_data = xmltodict.parse(xmltodict.parse(resp)['soap:Envelope'][
        'soap:Body']['RequestResponse']['RequestResult'])
    # xml 转 dict
    xmlt_data = (json.loads(
        (json.dumps(xmlt_data, ensure_ascii=False, indent=2)).replace('@', '')))

    # 代码健壮性之首要！
    def judgment(keys, data=xmlt_data, default=[]):
        try:
            if isinstance(keys, str)and isinstance(data, dict):
                return data.get(keys, default)
            if isinstance(keys, list)and isinstance(data, dict):
                if not len(keys) == 1:
                    if judgment(keys[0], data=data):
                        return judgment(keys[1:], data=judgment(keys[0], data=data))
                else:
                    return judgment(keys[-1], data=data)
            else:
                return []
        except KeyError as e:
            return []
    access_count = judgment(['Response', 'Header', 'AccessCount'])
    hotel_ids = judgment(
        ['Response', 'GetHotelInfoIncrementResponse', 'HotelInfoChangeList', 'HotelID'])
    pprint(xmlt_data)
#     if hotel_ids:
#         if not isinstance(hotel_ids, list):
#             hotel_ids = [hotel_ids]
#         page_index += 1
#         get_hotel_info_increment.delay(city_id, page_index)
#         # print(city_id, hotel_ids)
#         for hotel_id in hotel_ids:
#             hotel_message_fetch(hotel_id)  # async
#         return['city', city_id, 'page_index', page_index - 1, 'updating']
#     else:
#         return ['city', city_id, 'update increment is over']


In [763]:
get_hotel_info_increment(7)

{'Response': {'GetHotelInfoIncrementResponse': {'HotelInfoChangeList': {'HotelID': ['439773',
                                                                                    '467216',
                                                                                    '1137296',
                                                                                    '823433',
                                                                                    '1702098',
                                                                                    '5982552',
                                                                                    '443641',
                                                                                    '669757',
                                                                                    '792745',
                                                                                    '371190',
                                                         

In [756]:
get_hotel_info_increment(7,11)

{'Response': {'GetHotelInfoIncrementResponse': {'HotelInfoChangeList': {'HotelID': ['439773',
                                                                                    '371190',
                                                                                    '467216',
                                                                                    '1137296',
                                                                                    '823433',
                                                                                    '1702098',
                                                                                    '5982552',
                                                                                    '443641',
                                                                                    '669757',
                                                                                    '792745',
                                                         